# Задание 1

Скрипт для асинхронной обкачки урлов

Написать скрипт для обкачки списка урлов с возможностью задавать количество одновременных запросов, используя асинхронное программирование. Клиент можно использовать любой, например, из aiohttp.

In [2]:
!pip install aiohttp
!pip install nest-asyncio 

In [6]:
import aiohttp
import asyncio
from aiohttp import ClientSession

# Лечилка для Юпитера
import nest_asyncio
nest_asyncio.apply()

async def fetch_url(session: ClientSession, url: str):
    """Асинхронная функция для получения содержимого URL."""
    try:
        async with session.get(url) as response:
            status = response.status
            content = await response.text()
            print(f"Fetched {url} with status {status}")
            return url, status, content
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return url, None, None

async def fetch_all_urls(urls: list, max_concurrent: int):
    """Асинхронная функция для обкачки списка URL."""
    semaphore = asyncio.Semaphore(max_concurrent)  # Ограничение на количество одновременных запросов

    async def fetch_with_semaphore(session, url):
        async with semaphore:
            return await fetch_url(session, url)

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_with_semaphore(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        return results

if __name__ == "__main__":
    urls_to_fetch = [
        "https://example.com",
        "https://nohello.net/",
        "https://google.com",
        "https://vk.com/"
        "https://ya.ru",
        "https://youtube.com"
    ]

    max_concurrent_requests = 3 # Задаем количество одновременных запросов

    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(fetch_all_urls(urls_to_fetch, max_concurrent_requests))

    # Обработка результатов
    for url, status, content in results:
        print(f"URL: {url}, Status: {status}, Content Length: {len(content) if content else 'N/A'}")


Fetched https://nohello.net/ with status 200
Fetched https://google.com with status 200
Failed to fetch https://vk.com/https://ya.ru: Cannot connect to host vk.com:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')]
Fetched https://example.com with status 200
Fetched https://youtube.com with status 200
URL: https://example.com, Status: 200, Content Length: 1256
URL: https://nohello.net/, Status: 200, Content Length: 3583
URL: https://google.com, Status: 200, Content Length: 21416
URL: https://vk.com/https://ya.ru, Status: None, Content Length: N/A
URL: https://youtube.com, Status: 200, Content Length: 582881


# Задание 2

Сравнение использования weakref и слотов

Нужно придумать свои типы с несколькими атрибутами:

- класс с обычными атрибутами
- класс со слотами
- класс с атрибутами weakref

Для каждого класса создается большое число экземпляров и замеряется (сравнивается):
- время создания пачки экземпляров
- время чтения/изменения атрибутов

Результаты замеров оформляются скриншотами c описанием и выводом.

In [4]:
import time
import weakref

# Класс с обычными атрибутами
class RegularClass:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

# Класс со слотами
class SlotsClass:
    __slots__ = ('a', 'b', 'c')

    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

# Класс с поддержкой weakref
class WeakrefClass:
    __slots__ = ('a', 'b', 'c', '__weakref__')

    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

def measure_creation(cls, n):
    """Замер времени создания n экземпляров."""
    start = time.time()
    instances = [cls(i, i + 1, i + 2) for i in range(n)]
    end = time.time()
    return end - start, instances

def measure_attr_access(instances):
    """Замер времени чтения/изменения атрибутов."""
    start = time.time()
    for obj in instances:
        _ = obj.a  # чтение
        obj.a += 1  # изменение
    end = time.time()
    return end - start

def main():
    n = 10**6  # количество экземпляров

    # Замеры для RegularClass
    reg_time, reg_instances = measure_creation(RegularClass, n)
    reg_access_time = measure_attr_access(reg_instances)

    # Замеры для SlotsClass
    slots_time, slots_instances = measure_creation(SlotsClass, n)
    slots_access_time = measure_attr_access(slots_instances)

    # Замеры для WeakrefClass
    weak_time, weak_instances = measure_creation(WeakrefClass, n)
    weak_access_time = measure_attr_access(weak_instances)

    # Вывод результатов
    print(f"RegularClass: создание {reg_time:.2f} с, доступ {reg_access_time:.2f} с")
    print(f"SlotsClass: создание {slots_time:.2f} с, доступ {slots_access_time:.2f} с")
    print(f"WeakrefClass: создание {weak_time:.2f} с, доступ {weak_access_time:.2f} с")

if __name__ == "__main__":
    main()

RegularClass: создание 0.75 с, доступ 0.04 с
SlotsClass: создание 0.49 с, доступ 0.04 с
WeakrefClass: создание 0.60 с, доступ 0.03 с
